In [49]:
import sqlite3
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time
import difflib
import overpy
import warnings
import numpy as np
import unicodedata

warnings.filterwarnings("ignore")


def esta_en_villa_maria(lat, lon):
    return (-33 <= lat <= -29) and (-65 <= lon <= -63)

geolocator = Nominatim(user_agent="vm_dengue_mapper", timeout=20)
# Función con backoff progresivo y control de errores
def geocode_direccion(direccion, max_retries=5):
    delay = 2
    for intento in range(1, max_retries + 1):
        try:
            # print(f"[BUSCANDO] {direccion} (intento {intento})")
            time.sleep(delay)
            return geolocator.geocode(direccion)
        except (GeocoderTimedOut, GeocoderUnavailable) as e:
            print(f"[REINTENTO] {direccion} → {str(e)}")
            time.sleep(delay)
            delay *= 2  # Exponencial: 2, 4, 8...
        except Exception as e:
            print(f"[FALLO FATAL] {direccion} → {str(e)}")
            break
    return None



# — Obtención de la lista de calles con Overpass —
api = overpy.Overpass()
query = """
area["boundary"="administrative"]["name"="Villa María"]["admin_level"="8"]->.searchArea;
way["highway"]["name"](area.searchArea);
out;
"""
result = api.query(query)
calles = sorted({ way.tags["name"] for way in result.ways if "name" in way.tags })
import unicodedata
def normalize(texto: str) -> str:
    """
    Quita acentos y otros diacríticos, pero preserva la ñ, y convierte a mayúsculas.
    """
    # 1. Descomponer en NFKD (“ñ” para ñ, “á” para á, etc.)
    nfkd = unicodedata.normalize('NFKD', texto)
    out_chars = []
    prev = None

    for c in nfkd:
        if unicodedata.combining(c):
            # Sólo dejamos la tilde (~) si va sobre una 'n' o 'N'
            if c == '\u0303' and prev and prev.lower() == 'n':
                out_chars.append(c)
            # resto de diacríticos los descartamos
        else:
            out_chars.append(c)
            prev = c

    # 2. Unir, pasar a mayúsculas y recomponer (NFC) para que 'N'+~ vuelva a ser 'Ñ'
    result = ''.join(out_chars).upper()
    return unicodedata.normalize('NFC', result)

def buscar_calle_similar(texto, lista_calles, n=5, cutoff=0.6):
    # Normalizamos lista de calles
    originales = lista_calles
    normalized_calles = [normalize(c) for c in originales]
    target = normalize(texto)
    # Buscamos coincidencias en la forma normalizada
    matches = difflib.get_close_matches(target, normalized_calles, n=n, cutoff=cutoff)
    # Reconstruimos los nombres con acento original
    return [ originales[normalized_calles.index(m)] for m in matches ]

def procesar_coordenadas(df, lon_col='lon', lat_col='lat', wkt_col='WKT'):
    # 1. Detectar filas donde 'lon' contiene dos valores separados por coma
    mask = df[lon_col].astype(str).str.contains(',', na=False)

    # 2. Si no hay ninguna fila con coma, devolvemos el df sin cambios
    if not mask.any():
        return df

    # 3. Split y renombrado de columnas temporales

    
    split = (
        df.loc[mask, lon_col]
          .astype(str)
          .str.split(',', expand=True)
          .rename(columns={0: lat_col, 1: lon_col})
    )

    # 4. Limpiar espacios y convertir a float
    split[lat_col] = split[lat_col].str.replace('"','').str.strip().astype(float)
    split[lon_col] = split[lon_col].str.replace('"','').str.strip().astype(float)

    # 5. Asignar de vuelta al df original
    df.loc[mask, lat_col] = split[lat_col]
    df.loc[mask, lon_col] = split[lon_col]

    # 6. Generar la columna WKT
    df[wkt_col] = df.apply(
        lambda row: f"POINT({row[lon_col]} {row[lat_col]})"
                    if pd.notna(row[lon_col]) and pd.notna(row[lat_col])
                    else pd.NA,
        axis=1
    )

    return df
def geocode_intersection_op(street1, street2, area_name="Villa María"):
    query = f"""
    area["boundary"="administrative"]["name"="{area_name}"]->.a;
    way(area.a)["highway"]["name"="{street1}"]->.w1;
    way(area.a)["highway"]["name"="{street2}"]->.w2;
    node(w1.w2)->.n;
    out body n 1;
    """
    try:
        res = api.query(query)
        if res.nodes:
            n = res.nodes[0]
            return float(n.lat), float(n.lon)
    except Exception:
        pass
    return None
def is_intersection(addr):
    a = addr.lower()
    return " y " in a or " esq. " in a or " esq " in a

def split_intersection(addr):
    low = addr.lower()
    if " y " in low:
        parts = low.split(" y ", 1)
    elif " esq. " in low:
        parts = low.split(" esq. ", 1)
    else:
        parts = low.split(" esq ", 1)
    print(parts)
    return parts[0].strip().capitalize(), parts[1].strip().capitalize()

import numpy as np
import re


def asignar_location(location,df,idx,dir_raw,mensaje=0):
    lat = location.latitude
    lon = location.longitude
    
    if not esta_en_villa_maria(lat, lon):
        print(f"[ERROR GEO] Coordenadas fuera de Villa María: {dir_raw} → Lat: {lat}, Lon: {lon}")


    wkt = f"POINT({lon} {lat})"
    df.at[idx, 'lat'] = lat
    df.at[idx, 'lon'] = lon
    df.at[idx, 'WKT'] = wkt
    if mensaje==1:
        print(f"[ENCONTRADA] Dirección 2: {dir_raw} → Lat: {lat}, Lon: {lon}")
    else:
        print(f"[ENCONTRADA] Dirección: {dir_raw} → Lat: {lat}, Lon: {lon}")
    return df
def evaluar_direccion(df,columna_direccion="DIRECCION"):
    
    # Inicializar geocoder
   
    if "lon" not in df.columns:
        df["lon"]=np.nan
    # Filas con lon vacía
    faltantes = df[df['lon'].isna()].copy()

    # Si no hay una columna de dirección clara, asumimos una posible:

    contador=0

    reemplazos={'TTE.':'TENIENTE','INT':'INTENDENTE',"MEJICO":"MEXICO",'Antonio Hosch':'Enrique Antonio Hoch','Enrique A. Hoch':'Enrique Antonio Hoch',
                'T. PEÑA':"INTENDENTE PEÑA","EEUU":"ESTADOS UNIDOS",'BV ALVEAR':'Boulevard Marcelo T. de Alvear','CTDA. ': ""}
    for idx, row in faltantes.iterrows():
        contador += 1
        dir_raw = row[columna_direccion]
        print(contador)
        dir_raw=dir_raw.upper()
        if not isinstance(dir_raw, str) or dir_raw.strip() == "" \
        or "ZONA RURAL" in dir_raw.upper() \
        or "NO HAY" in dir_raw.upper() or "SIN D" in dir_raw.upper() or "NO SE E" in dir_raw.upper() or r"S/D" in dir_raw.upper() \
        or r"NO TIENE" in dir_raw.upper():
            continue

        for k, v in reemplazos.items():
            if k in dir_raw:
                dir_raw = dir_raw.replace(k, v, 1)  # solo el primero que aparezca
   

        base = dir_raw + ", Villa María, Córdoba, Argentina"

        location = geocode_direccion(base)

        if location:
            df=asignar_location(location,df,idx,dir_raw)



        else:
            # Extraemos el nombre bruto de la calle
            street = dir_raw.split(',')[0].strip()

            similares = buscar_calle_similar(street, calles, n=5, cutoff=0.6)               


            if similares:
                for calle in similares:
                    intento = f"{calle}, Villa María, Córdoba, Argentina"
                    loc2 = geocode_direccion(intento)
                    if loc2 and esta_en_villa_maria(loc2.latitude, loc2.longitude):
                        df.at[idx, 'lat'] = loc2.latitude
                        df.at[idx, 'lon'] = loc2.longitude
                        df.at[idx, 'WKT'] = f"POINT({loc2.longitude} {loc2.latitude})"
                        print(f"[ENCONTRADA POR SIMILARIDAD] {street} → {calle}: {intento} con LON {loc2.longitude} y LAT {loc2.latitude}")
                        break

                    # 2) Si no hubo match y es posible esquina, lo intentamos por Overpass

            elif is_intersection(dir_raw):
                print("Intersección",dir_raw)
                street1, street2 = split_intersection(dir_raw)
                location = geocode_direccion(street2)
                if location:
                    df=asignar_location(location,df,idx,street2)
                else:
                    inter = geocode_intersection_op(re.sub(r'\d+', '', street1), re.sub(r'\d+', '', street2))
                    print(inter,street1, street2)
                    if inter and esta_en_villa_maria(*inter):
                        lat, lon = inter
                        df.at[idx, 'lat'], df.at[idx, 'lon'] = lat, lon
                        df.at[idx, 'WKT'] = f"POINT({lon} {lat})"
                        print(f"[ENCONTRADA POR INTERSECCION] {street} → {inter}: con LON {loc2.longitude} y LAT {loc2.latitude}")
                        continue
                        

            else:
                print(f"[NO ENCONTRADA] Ninguna similar válida para '{street}'")

    return(df)






In [50]:

if 'df_casos_conf' not in locals():   # existe en el scope local

    ruta_excel = r"Datos\Casos\No utiles\notificaciones dengue Villa María 23-24.xlsx"
    df0 = pd.read_excel(ruta_excel)

    ruta_excel = r"Datos\Casos\No utiles\Notificaciones dengue Villa María 24-25.xlsx"
    df1 = pd.read_excel(ruta_excel)
    df_casos=pd.concat([df0,df1])

    cols_utiles=['sexo',"edad_diagnostico","calle_domicilio","numero_domicilio","fecha_apertura",'clasificacion_manual']
    df_casos=df_casos[cols_utiles]

    # df=procesar_coordenadas(df)
    df_casos_conf=df_casos.loc[(df_casos["clasificacion_manual"].str.contains("confirmado"))|
                            (df_casos["clasificacion_manual"].str.contains("Caso de Dengue en brote con laboratorio"))]

    df_casos_conf["DIRECCION"]=df_casos_conf["calle_domicilio"]+" "+df_casos_conf["numero_domicilio"]+", Villa María, Córdoba"
    df_casos_conf=df_casos_conf.drop(["calle_domicilio","numero_domicilio","clasificacion_manual"],axis=1)
    df_casos_conf.head()





ruta_excel_cc = r"Datos\Casos\No utiles\Casos Direccion Correcta.xlsx"
casos_correctos = pd.read_excel(ruta_excel_cc)



df_sacar=pd.merge(df_casos_conf,casos_correctos[["DIRECCION","WKT"]],on="DIRECCION",how="left")
lista=list(casos_correctos.loc[~casos_correctos["WKT"].isnull(),"DIRECCION"])
df_revisar=df_casos_conf.loc[~df_casos_conf["DIRECCION"].isin(lista)]
vacios=df_casos_conf.loc[df_casos_conf["DIRECCION"].isnull()]
len(df_casos_conf),len(df_casos_conf)-len(lista)-len(vacios)

df2=evaluar_direccion(df_revisar)
df2

1
[ENCONTRADA] Dirección: LA PAMPA 2247, VILLA MARÍA, CÓRDOBA → Lat: -32.4094327, Lon: -63.2253577
2
[ENCONTRADA] Dirección: INTENDENTE PEÑA 8710, VILLA MARÍA, CÓRDOBA → Lat: -32.4001128, Lon: -63.2357674
3
[NO ENCONTRADA] Ninguna similar válida para 'RUTA 9 KM 562 0'
4
[NO ENCONTRADA] Ninguna similar válida para 'NICOLÁS AVELLANEDA      SAN PABLO 2650'
5
6
[NO ENCONTRADA] Ninguna similar válida para 'NOLBERTO GUIZZO 1058'
7
[NO ENCONTRADA] Ninguna similar válida para 'S.J. BAUTISTA - LAS HERAS 2546'
8
[NO ENCONTRADA] Ninguna similar válida para 'ISLAS BALEARES 774'
9
[ENCONTRADA] Dirección: ANTUÑA 65, VILLA MARÍA, CÓRDOBA → Lat: -32.4038689, Lon: -63.2302257
10
[NO ENCONTRADA] Ninguna similar válida para 'R. JUAREZ 1459'
11
[NO ENCONTRADA] Ninguna similar válida para 'RAMIRO SUAREZ BARRIO ALMIRANTE BROWN 1392'
12
[ENCONTRADA] Dirección: Boulevard Marcelo T. de Alvear 411, VILLA MARÍA, CÓRDOBA → Lat: -32.412156, Lon: -63.2352895
13
[NO ENCONTRADA] Ninguna similar válida para 'PERAZOLLO

,sexo,edad_diagnostico,fecha_apertura,DIRECCION,lon,lat,WKT
443,F,56,2024-04-08,"CTDA. LA PAMPA 2247, Villa María, Córdoba",-63.225358,-32.409433,POINT(-63.2253577 -32.4094327)
518,F,55,2024-04-11,"T. Peña 8710, Villa María, Córdoba",-63.235767,-32.400113,POINT(-63.2357674 -32.4001128)
584,F,52,2024-03-12,"RUTA 9 KM 562 0, Villa María, Córdoba",NaN,NaN,NaN
694,F,50,2024-04-08,"Nicolás Avellaneda San Pablo 2650, Villa ...",NaN,NaN,NaN
704,F,49,2024-03-08,"Zona Rural Villa Maria-Tio Pujio -, Villa Marí...",NaN,NaN,NaN
...,...,...,...,...,...,...,...
3877,M,0,2024-03-23,"Raeson 1982, Villa María, Córdoba",NaN,NaN,NaN
3884,M,75,2024-04-22,"BV ALVEAR 56, Villa María, Córdoba",-63.238636,-32.408964,POINT(-63.2386357 -32.408964)
3895,M,77,2024-03-17,"PROL. ALVEAR 3400, Villa María, Córdoba",NaN,NaN,NaN
3914,M,48,2024-02-07,"Zona Rural Ruta 9 Km 548 -, Villa María, Córdoba",NaN,NaN,NaN


In [51]:
df44=df2.loc[~df2["lon"].isnull()]
ruta_excel_cc = r"Datos\Casos\No utiles\Casos Direccion Correcta.xlsx"
df_final=pd.concat([casos_correctos,df44]).reset_index(drop=True)
df_final.to_excel(ruta_excel_cc, index=False)

In [52]:
import winsound

# Beep sencillo: frecuencia 1 kHz durante 500 ms

def hacer_sonido():
    winsound.Beep(1000, 1300)

hacer_sonido()

In [53]:
import overpy

api = overpy.Overpass()
query = """
area["boundary"="administrative"]["name"="Villa María"]["admin_level"="8"]->.searchArea;
way["highway"]["name"](area.searchArea);
out;
"""
result = api.query(query)

# extraer nombres únicos
calles = sorted({ way.tags["name"] for way in result.ways if "name" in way.tags })
calles

['1 de Mayo',
 '12 de Octubre',
 '17 de Agosto',
 '2 de Abril',
 '20 de Junio',
 '25 de Mayo',
 '27 de Setiembre',
 '30 de Junio',
 '9 de Julio',
 'A. Yupanqui',
 'Abelino Antuña',
 'Abraham Ruiz',
 'Acacia',
 'Acapulco',
 'Aconcagua',
 'Aconquija',
 'Adrián Otero',
 'Aguirre Cámara',
 'Agustin Tosco',
 'Agustín Tosco',
 'Agustín Álvarez',
 'Aimé Painé',
 'Alaska',
 'Alberdi',
 'Aldo Serrano',
 'Alejandro Daniel Mie',
 'Alejandro Errazquin',
 'Alejandro Voglio',
 'Alfonsina Storni',
 'Aluminé',
 'Amalia C. Figueredo',
 'América',
 'Antofalla',
 'Antonio Berni',
 'Antártida Argentina',
 'Ara 17 de Octubre',
 'Araucanos',
 'Arenales',
 'Arequipa',
 'Arica',
 'Artigas',
 'Arturo M. Bas',
 'Ascasubi',
 'Asunción',
 'Atahualpa Yupanqui',
 'Atilio López',
 'Av María Elena Walsh',
 'Av. Los Algarrobos',
 'Avellaneda',
 'Avenida Aguará Guazú',
 'Avenida Arturo Jauretche',
 'Avenida Bruno Ceballos',
 'Avenida Cardenal Amarillo',
 'Avenida Costanera',
 'Avenida De Los Pueblos Aborígenes',
 'Aven

In [54]:

api = overpy.Overpass()
query = """
area["boundary"="administrative"]["name"="Villa Nueva"]["admin_level"="8"]->.searchArea;
way["highway"]["name"](area.searchArea);
out;
"""
result = api.query(query)

# extraer nombres únicos
calles = sorted({ way.tags["name"] for way in result.ways if "name" in way.tags })
calles

['10 de Junio',
 '12 de Octubre',
 '2 de Abril',
 '2 de Setiembre',
 '25 de Mayo',
 '26 de Septiembre',
 '31 de Octubre',
 '4 de Septiembre',
 '5 de Julio',
 '8 de Octubre',
 '9 de Julio',
 'A. Moreno',
 'Alberdi',
 'Alfredo Palacios',
 'Alheli',
 'Alicia Moreau de Justo',
 'Almirante Brown',
 'Alvear',
 'Ambrosio Olmos',
 'Antártida Argentina',
 'Atacama',
 'Atahualpa Yupanqui',
 'Avellaneda',
 'Avenida Arturo Humberto Illia',
 'Avenida C. Atlántica',
 'Avenida Carranza',
 'Avenida Cervantes',
 'Avenida Costanera',
 'Avenida Gardenias',
 'Avenida Henry Néstle',
 'Avenida Juan Domingo Peron',
 'Avenida La Pichona',
 'Avenida Las Fresias',
 'Avenida Las Violetas',
 'Avenida Los Arrayanes',
 'Avenida Los Ceibos',
 'Avenida Los Gigantes',
 'Avenida Los Lirios',
 'Avenida Patria',
 'Avenida Tilcara',
 'Azucenas',
 'Barberis',
 'Bartolomé Mitre',
 'Belgrano',
 'Bolivar',
 'Bosque Monte Blanco',
 'Bosque de Araucarias',
 'Bosque de Caldén',
 'Bosque de Hayas',
 'Bosque de Pataia',
 'Bosque d